In [4]:
import pandas as pd
import plotly.express as px
from datetime import datetime
from zoneinfo import ZoneInfo

# -------------------------
# LOAD DATA
# -------------------------
df = pd.read_csv("googleplaystore.csv")


# -------------------------
# DATA CLEANING
# -------------------------

# Clean Installs
df["Installs"] = (
    df["Installs"]
    .astype(str)
    .str.replace("+", "", regex=False)
    .str.replace(",", "", regex=False)
)
df["Installs"] = pd.to_numeric(df["Installs"], errors="coerce")

# Clean Reviews & Rating
df["Reviews"] = pd.to_numeric(df["Reviews"], errors="coerce")
df["Rating"] = pd.to_numeric(df["Rating"], errors="coerce")

# Convert Size to MB
def size_to_mb(x):
    try:
        if isinstance(x, str):
            if "M" in x:
                return float(x.replace("M", ""))
            elif "k" in x:
                return float(x.replace("k", "")) / 1024
        return None
    except:
        return None

df["Size_MB"] = df["Size"].apply(size_to_mb)

# -------------------------
# APPLY FILTERS
# -------------------------

categories_required = [
    "Game", "Beauty", "Business", "Comics",
    "Communication", "Dating", "Entertainment",
    "Social", "Events"
]

filtered = df[
    (df["Rating"] > 3.5) &
    (df["Category"].isin(categories_required)) &
    (df["Reviews"] > 500) &
    (~df["App"].str.contains("S", na=False)) &
    (df["Sentiment Subjectivity"] > 0.5) &
    (df["Installs"] > 50000)
]

filtered = filtered.dropna(subset=["Size_MB"])

# -------------------------
# TRANSLATIONS
# -------------------------

translations = {
    "Beauty": "सौंदर्य",          # Hindi
    "Business": "வணிகம்",        # Tamil
    "Dating": "Partnersuche"     # German
}

filtered["Category_Display"] = filtered["Category"].replace(translations)

# -------------------------
# TIME CONDITION (5PM–7PM IST)
# -------------------------

current_time = datetime.now(ZoneInfo("Asia/Kolkata"))
current_hour = current_time.hour

print("Current IST Time:", current_time.strftime("%Y-%m-%d %H:%M:%S"))

if 17 <= current_hour < 19 and not filtered.empty:

    fig = px.scatter(
        filtered,
        x="Size_MB",
        y="Rating",
        size="Installs",
        color="Category_Display",
        hover_name="App",
        size_max=60,
        title="App Size vs Rating (Bubble = Installs)\nVisible Only Between 5 PM IST and 7 PM IST"
    )

    # Highlight Game Category in Pink
    for trace in fig.data:
        if trace.name == "Game":
            trace.marker.color = "pink"

    fig.update_layout(
        xaxis_title="App Size (MB)",
        yaxis_title="Average Rating",
        template="plotly_white"
    )

    fig.show()

else:
    print("⚠ Dashboard Locked")
    print("Bubble Chart is visible ONLY between 5 PM IST and 7 PM IST.")


KeyError: 'Sentiment Subjectivity'